In [21]:
import zeep
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

wsdl = 'http://sistemas.cvm.gov.br/webservices/Sistemas/SCW/CDocs/WsDownloadInfs.asmx?WSDL'
client = zeep.Client(wsdl=wsdl)
soap_header = client.service.Login('2293', '8700')
soap_header['header']

In [23]:
header = {
    'sessaoIdHeader': {
        'Guid': '2d572807-3eba-4fde-8a9c-8b1bfd503a43',
        'IdSessao': 822240093L,
        '_attr_1': None
    }
}


result_docs = client.service.solicAutorizDownloadArqComptc(_soapheaders=header, iCdTpDoc='209', strDtComptcDoc='2016-12-01', strMotivoAutorizDownload='Download documentos, mes 12')

In [3]:
url_docs = result_docs['body']['solicAutorizDownloadArqComptcResult']

In [4]:
from StringIO import StringIO
from zipfile import ZipFile
from urllib import urlopen

url = urlopen(url_docs)
zipfile = ZipFile(StringIO(url.read()))
lines = None
for name in zipfile.namelist():
    lines = zipfile.open(name).readlines()

In [5]:
import xmltodict
data = xmltodict.parse(''.join(lines))

In [6]:
import json
for informe in data['ROOT']['INFORMES']['INFORME_DIARIO']:
    print json.dumps(informe, ensure_ascii=False)
    break

{"CNPJ_FDO": "18600016000109", "DT_COMPTC": "2016-12-01", "VL_TOTAL": "82651948,66", "VL_QUOTA": "14,144225300000", "PATRIM_LIQ": "82618159,38", "CAPTC_DIA": "0,00", "RESG_DIA": "0,00", "NR_COTST": "1"}


In [13]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.CVM
informes_collection = db.informes
for informe in data['ROOT']['INFORMES']['INFORME_DIARIO']:
    informes_collection.insert_one(informe)
    